<a href="https://colab.research.google.com/github/dhyannn/LLM/blob/main/Dhyan_514_LLMlab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForTokenClassification
from transformers import DataCollatorForTokenClassification
from datasets import load_dataset, load_metric

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Load the dataset
dataset = load_dataset("conll2003")
label_list = dataset['train'].features['ner_tags'].feature.names

# Tokenize the dataset
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if word_id is None else label[word_id] for word_id in word_ids]
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Load the model
model = TFBertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_list))

# Data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Metrics
metric = load_metric("seqeval")

# Training arguments
batch_size = 8
num_epochs = 3
learning_rate = 2e-5

# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Define the loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Define the metric
def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [label_list[l] for l in label if l != -100]
        for label in labels
    ]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Convert datasets to tf.data.Dataset
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'labels'],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'labels'],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

# Compile the model
model.compile(optimizer=optimizer, loss=loss)

# Train the model
model.fit(train_dataset, validation_data=validation_dataset, epochs=num_epochs)

# Evaluate the model
predictions, labels = model.predict(validation_dataset)
compute_metrics((predictions, labels))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

ValueError: word_ids() is not available when using non-fast tokenizers (e.g. instance of a `XxxTokenizerFast` class).

In [ ]:
data.head(20)

,text,labels
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...
5,Mr. Egeland said the latest figures show 1.8 m...,B-per I-per O O O O O O O O O O O O O O O O O ...
6,He said last week 's tsunami and the massive u...,O O O O O O O O O O O O O O O O O O B-geo O B-...
7,"Some 1,27,000 people are known dead .",O O O O O O O
8,"Aid is being rushed to the region , but the U....",O O O O O O O O O O B-geo O O O O O O O O O O ...
9,Lebanese politicians are condemning Friday 's ...,B-gpe O O O B-tim O O O O O O O O B-geo O O O ...


In [ ]:
import pandas as pd
data=pd.read_csv('/content/ner.csv')
data.head()

,text,labels
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...


In [ ]:
texts = data['text'].apply(lambda x: x.split()).values
labels = data['labels'].apply(lambda x: x.split()).values

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
label_list = list(set([label for sublist in labels for label in sublist]))
label_map = {label: i for i, label in enumerate(label_list)}

In [ ]:
encoded_labels = []
for label in labels:
  encoded_labels.append([label_map[l] for l in label])

In [ ]:
encoded_labels = []
for label in labels:
  encoded_labels.append([label_map[l] for l in label])

In [ ]:
import numpy as np
def preprocess_data(texts, labels):
    input_ids = []
    attention_masks = []
    label_ids = []

    for i in range(len(texts)):
        # Tokenize input texts
        tokenized_input = tokenizer(texts[i], is_split_into_words=True, truncation=True, padding='max_length', max_length=128)
        input_ids.append(tokenized_input['input_ids'])
        attention_masks.append(tokenized_input['attention_mask'])

        # Align labels
        label_id = [-100] * len(tokenized_input['input_ids'])
        label_index = 0
        for j in range(1, len(tokenized_input['input_ids']) - 1):  # Skip [CLS] and [SEP]
            if tokenized_input['input_ids'][j] == tokenizer.pad_token_id:
                break
            if label_index < len(labels[i]):
                label_id[j] = labels[i][label_index]
                label_index += 1

        label_ids.append(label_id)

    return {
        'input_ids': np.array(input_ids),
        'attention_mask': np.array(attention_masks),
        'labels': np.array(label_ids)
    }

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, encoded_labels, test_size=0.2, random_state=42)
train_data = preprocess_data(train_texts, train_labels)
test_data = preprocess_data(test_texts, test_labels)

In [ ]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': train_data['input_ids'], 'attention_mask': train_data['attention_mask']},
    train_data['labels']
)).shuffle(1000).batch(8).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': test_data['input_ids'], 'attention_mask': test_data['attention_mask']},
    test_data['labels']
)).batch(8).prefetch(tf.data.AUTOTUNE)

In [ ]:
from transformers import TFBertForTokenClassification
model = TFBertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_list))

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
model.fit(train_dataset, epochs=3, validation_data=test_dataset)


Epoch 1/3
4796/4796 [==============================] - 1142s 229ms/step - loss: nan - accuracy: 0.0033 - val_loss: nan - val_accuracy: 0.0033
Epoch 2/3
4796/4796 [==============================] - 1088s 227ms/step - loss: nan - accuracy: 0.0033 - val_loss: nan - val_accuracy: 0.0033
Epoch 3/3
4796/4796 [==============================] - 1083s 226ms/step - loss: nan - accuracy: 0.0033 - val_loss: nan - val_accuracy: 0.0033


In [ ]:
def predict_ner(text):
    tokenized_input = tokenizer(text.split(), return_tensors="tf", truncation=True, padding=True, max_length=128, is_split_into_words=True)
    outputs = model(tokenized_input)
    predictions = tf.argmax(outputs.logits, axis=-1).numpy()[0]
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'].numpy()[0])
    predicted_labels = [label_list[pred] for pred in predictions]
    return list(zip(tokens, predicted_labels))

In [ ]:
text = input("Enter text for NER: ")
ner_tags = predict_ner(text)
for word, label in ner_tags:
  print(f"{word}: {label}")

Enter text for NER: Alice went to New York and met Bob there.
[CLS]: O
Alice: B-PER
went: O
to: O
New: B-LOC
York: I-LOC
and: O
met: O
Bob: B-PER
there: O
.: O
[SEP]: O


In [ ]:
text = input("Enter text for NER: ")
ner_tags = predict_ner(text)
for word, label in ner_tags:
  print(f"{word}: {label}")

Enter text for NER: feeling good
[CLS]: B-tim
feeling: B-tim
good: B-tim
[SEP]: B-tim


In [ ]:
text = input("Enter text for NER: ")
ner_tags = predict_ner(text)
for word, label in ner_tags:
  print(f"{word}: {label}")

Enter text for NER: The global financial crisis has left Iceland
[CLS]: B-tim
The: B-tim
global: B-tim
financial: B-tim
crisis: B-tim
has: B-tim
left: B-tim
Iceland: B-tim
[SEP]: B-tim


In [ ]:
text = input("Enter text for NER: ")
ner_tags = predict_ner(text)
for word, label in ner_tags:
  print(f"{word}: {label}")

Enter text for NER: happy 
[CLS]: B-tim
happy: B-tim
[SEP]: B-tim
